# Bayesian Logistic Regression 


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.random.seed(42)

In [4]:

df = pd.read_csv("atp_matches_2010_2024_missing_handled.csv")

print("Dataset Head:")
print(df.head())


print("\nColumns in Dataset:")
print(df.columns)

# Encode categorical variable: surface
modelLR_df = pd.get_dummies(df, columns=["surface"], drop_first=True)

# Convert bool to int for one-hot encoded columns
for col in ["surface_Grass", "surface_Hard"]:
    modelLR_df[col] = modelLR_df[col].astype(int)

modelLR_df["rank_diff"] = modelLR_df["winner_rank"] - modelLR_df["loser_rank"]
modelLR_df["ace_diff"] = modelLR_df["w_ace"] - modelLR_df["l_ace"]
modelLR_df["df_diff"] = modelLR_df["w_df"] - modelLR_df["l_df"]
modelLR_df["svpt_diff"] = modelLR_df["w_svpt"] - modelLR_df["l_svpt"]
modelLR_df["1stIn_diff"] = modelLR_df["w_1stIn"] - modelLR_df["l_1stIn"]
modelLR_df["1stWon_diff"] = modelLR_df["w_1stWon"] - modelLR_df["l_1stWon"]
modelLR_df["2ndWon_diff"] = modelLR_df["w_2ndWon"] - modelLR_df["l_2ndWon"]
modelLR_df["SvGms_diff"] = modelLR_df["w_SvGms"] - modelLR_df["l_SvGms"]
modelLR_df["bpSaved_diff"] = modelLR_df["w_bpSaved"] - modelLR_df["l_bpSaved"]
modelLR_df["bpFaced_diff"] = modelLR_df["w_bpFaced"] - modelLR_df["l_bpFaced"]
modelLR_df["age_diff"] = modelLR_df["winner_age"] - modelLR_df["loser_age"]


feature_cols = [
    "rank_diff", "ace_diff", "df_diff", "svpt_diff", "1stIn_diff", "1stWon_diff",
    "2ndWon_diff", "SvGms_diff", "bpSaved_diff", "bpFaced_diff", "age_diff",
    "surface_Grass", "surface_Hard"  # Assuming Clay is the reference
    ]

# Target: 1 if winner is Player A (recorded winner), 0 if loser wins (flip for symmetry later)
modelLR_df["target"] = 1

# Create a symmetric dataset by flipping winner/loser
df_flipped = modelLR_df.copy()

df_flipped["rank_diff"] = -df_flipped["rank_diff"]
df_flipped["ace_diff"] = -df_flipped["ace_diff"]
df_flipped["df_diff"] = -df_flipped["df_diff"]
df_flipped["svpt_diff"] = -df_flipped["svpt_diff"]
df_flipped["1stIn_diff"] = -df_flipped["1stIn_diff"]
df_flipped["1stWon_diff"] = -df_flipped["1stWon_diff"]
df_flipped["2ndWon_diff"] = -df_flipped["2ndWon_diff"]
df_flipped["SvGms_diff"] = -df_flipped["SvGms_diff"]
df_flipped["bpSaved_diff"] = -df_flipped["bpSaved_diff"]
df_flipped["bpFaced_diff"] = -df_flipped["bpFaced_diff"]
df_flipped["age_diff"] = -df_flipped["age_diff"]
df_flipped["target"] = 0

# Combine original and flipped data
df_symmetric = pd.concat([modelLR_df, df_flipped], ignore_index=True)

# Step 2: Train-Test Split
df_symmetric["tourney_date"] = pd.to_datetime(df_symmetric["tourney_date"], format="%Y%m%d", errors="coerce")
df_symmetric = df_symmetric.dropna(subset=["tourney_date"])
df_symmetric["year"] = df_symmetric["tourney_date"].dt.year
train_data = df_symmetric[df_symmetric["year"] <= 2022]
test_data = df_symmetric[df_symmetric["year"] > 2022]

X_train = train_data[feature_cols].values
y_train = train_data["target"].values
X_test = test_data[feature_cols].values
y_test = test_data["target"].values

# Debug: Check array shape and type
print("\nX_train shape:", X_train.shape)
print("X_train dtype:", X_train.dtype)
print("X_test shape: ", X_test.shape)

# Verifying the shapes of the feature matrix and target vector
print("\nFeature Matrix Shape:", X.shape)
print("Target Vector Shape:", y.shape)

# Step 3: Normalize features
X_train_mean = np.nanmean(X_train, axis=0)
X_train_std = np.nanstd(X_train, axis=0)
X_train_std[X_train_std == 0] = 1e-10  # Avoid division by zero with small epsilon
X_train = np.where(np.isnan(X_train), 0, X_train)
X_train = (X_train - X_train_mean) / X_train_std
X_test = np.where(np.isnan(X_test), 0, X_test)
X_test = (X_test - X_train_mean) / X_train_std

# Add intercept term
X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])




Dataset Head:
  tourney_id tourney_name surface  draw_size tourney_level  tourney_date  \
0   2010-339     Brisbane    Hard         32             A      20100103   
1   2010-339     Brisbane    Hard         32             A      20100103   
2   2010-339     Brisbane    Hard         32             A      20100103   
3   2010-339     Brisbane    Hard         32             A      20100103   
4   2010-339     Brisbane    Hard         32             A      20100103   

   match_num  winner_id     winner_name winner_hand  ...  l_1stIn l_1stWon  \
0          1     104053    Andy Roddick           R  ...     34.0     29.0   
1         30     103285  Radek Stepanek           R  ...     27.0     14.0   
2         29     104053    Andy Roddick           R  ...     43.0     34.0   
3         28     103285  Radek Stepanek           R  ...     40.0     25.0   
4         27     104792    Gael Monfils           R  ...     50.0     38.0   

   l_2ndWon  l_SvGms l_bpSaved l_bpFaced  winner_rank winner

NameError: name 'feature_cols' is not defined

# Training the BLR Model

In [ ]:
# Define the sigmoid function (with clipping to prevent numerical issues)
def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

# Compute the negative log-posterior (loss) for Bayesian logistic regression.
def bayes_loss(X, y, beta, tau):
    """
    Computes the negative log-posterior (loss) for Bayesian Logistic Regression.
    The loss is the negative log-likelihood plus the regularization term from the Gaussian prior.
    
    Parameters:
        X (ndarray): Feature matrix with intercept column (shape: [n_samples, n_features+1])
        y (ndarray): Binary target vector (shape: [n_samples])
        beta (ndarray): Coefficient vector (including intercept) (shape: [n_features+1])
        tau (float): Prior standard deviation.
    
    Returns:
        loss (float): The computed loss.
    """
    # Compute predictions
    z = np.dot(X, beta)
    y_pred = sigmoid(z)
    
    # Negative log-likelihood (with epsilon added for numerical stability)
    nll = -np.mean(y * np.log(y_pred + 1e-10) + (1 - y) * np.log(1 - y_pred + 1e-10))
    
    # Regularization term: apply only to non-intercept coefficients (beta[1:])
    reg = (1 / (2 * tau**2)) * np.sum(beta[1:] ** 2)
    
    return nll + reg

# Compute the gradient of the negative log-posterior
def bayes_gradient(X, y, beta, tau):
    """
    Computes the gradient of the negative log-posterior for Bayesian Logistic Regression.
    
    Parameters:
        X (ndarray): Feature matrix with intercept column.
        y (ndarray): Binary target vector.
        beta (ndarray): Coefficient vector (including intercept).
        tau (float): Prior standard deviation.
    
    Returns:
        gradient (ndarray): The gradient vector.
    """
    z = np.dot(X, beta)
    y_pred = sigmoid(z)
    
    # Gradient from the negative log-likelihood
    grad = np.dot(X.T, (y_pred - y)) / len(y)
    
    # Add gradient from the regularization term (do not regularize intercept beta[0])
    reg_grad = np.concatenate(([0], beta[1:])) / (tau**2)
    
    return grad + reg_grad

# Gradient Descent for Bayesian Logistic Regression
def train_bayesian_lr(X, y, tau, learning_rate=0.01, n_iterations=5000):
    """
    Train Bayesian Logistic Regression using gradient descent.
    
    Parameters:
        X (ndarray): Feature matrix with intercept column.
        y (ndarray): Binary target vector.
        tau (float): Prior standard deviation.
        learning_rate (float): Learning rate for gradient descent.
        n_iterations (int): Number of iterations.
    
    Returns:
        beta (ndarray): Learned coefficient vector.
    """
    beta = np.zeros(X.shape[1])  # Initialize coefficients
    for i in range(n_iterations):
        grad = bayes_gradient(X, y, beta, tau)
        beta -= learning_rate * grad
        if i % 100 == 0:
            loss = bayes_loss(X, y, beta, tau)
            print(f"Iteration {i}, Loss: {loss:.4f}")
    return beta


# Set hyperparameter for the prior (tau)
tau = 1.0  # You can tune this value

# Train the Bayesian Logistic Regression model
beta_bayes = train_bayesian_lr(X_train, y_train, tau, learning_rate=0.01, n_iterations=5000)
print("Estimated coefficients (Bayesian LR):", beta_bayes)

# Prediction function (same as before)
def predict(X, beta):
    z = np.dot(X, beta)
    y_pred = sigmoid(z)
    return (y_pred >= 0.5).astype(int)

# Evaluate on the training set
y_train_pred_bayes = predict(X_train, beta_bayes)
train_accuracy_bayes = np.mean(y_train_pred_bayes == y_train)
print(f"\nTraining Accuracy (Bayesian LR): {train_accuracy_bayes:.2%}")

# Evaluate on the test set
y_test_pred_bayes = predict(X_test, beta_bayes)
test_accuracy_bayes = np.mean(y_test_pred_bayes == y_test)
print(f"Test Accuracy (Bayesian LR): {test_accuracy_bayes:.2%}")
